# Get Expression Data From AMP-PD Data on Terra
- **Author(s)** - Frank Grenn
- **Quick Description:** Run this on Terra. Query AMPPD data to get expression for the genes/transcripts and samples we want.

## Setup

In [ ]:
library(tidyr)
library(data.table)
# Load the tidyverse
library(tidyverse)

# Load ggplot2 for graphs
library(ggplot2)

# Load biqrquery for interacting with BigQuery
library(bigrquery)

# Load reticulate for calling the FireCloud Python API
library(reticulate)

In [ ]:
# Utility routine for printing a shell command before executing it
shell_do <- function(command) {
    print(paste('Executing: ', command))
    system(command, intern = TRUE)
}

# Utility routines for reading files from Google BigQuery
bq_query <- function(query) {
    # Return the contents of a query against BigQuery    
    return(bigrquery::bq_table_download(
        bigrquery::bq_project_query(BILLING_PROJECT_ID, query = query)))
}

# Initialize authorization for BigQuery operations
bigrquery::bq_auth(path=Ronaldo::getServiceAccountKey())

In [ ]:
WORKSPACE_BUCKET = 'gs://123'

In [ ]:
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')
WORKSPACE_NAMESPACE <- Sys.getenv('WORKSPACE_NAMESPACE')
WORKSPACE_NAME <- Sys.getenv('WORKSPACE_NAME')

print(BILLING_PROJECT_ID)
print(WORKSPACE_NAMESPACE)
print(WORKSPACE_NAME)



## Identify samples to use

In [ ]:
#pull the file containing the samples we want
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/chrY_meta.csv ."))


In [ ]:
meta <- fread("chrY_meta.csv")
print(dim(meta))

In [ ]:
#subset
samples_use <- (meta[which(meta$genetic_carrier==0 & meta$InfPop=="EUROPE" & meta$pheno!=-9),])
print(dim(samples_use))
print(head(samples_use))

## Query featureCounts Data

In [ ]:
#peek at the data
bq_query(str_glue("
  SELECT *
  FROM `amp-pd-research.2021_v2_5release_0510_transcriptomics.feature_counts`
LIMIT 10"))

In [ ]:
#want expression for baseline (BLM0T1) samples
samples_use$sample_id <- paste0(samples_use$iid,'-BLM0T1')

In [ ]:
#make a string to use in the query to get the samples we want
query_samples_string <- (paste(shQuote(c(samples_use$sample_id),type="cmd"),collapse=", "))
print(substr(query_samples_string,1,100))

In [ ]:
#get the chrY expression data for the samples
query_results <- bq_query(str_glue("
  SELECT participant_id, Geneid, Value, Chr
  FROM `amp-pd-research.2021_v2_5release_0510_transcriptomics.feature_counts`
  WHERE sample_id IN ({query_samples_string}) AND Chr LIKE 'chrY%'"))
print(dim(query_results))

In [ ]:
print(head(query_results))
print(length(unique(query_results$participant_id)))
print(length(unique(query_results$Geneid)))

In [ ]:
print(tail(query_results))

In [ ]:
#force it into a matrix
count_df <- query_results %>% select("participant_id","Geneid","Value")
count_matrix <- pivot_wider(count_df, names_from = participant_id, values_from = Value)
print(dim(count_matrix))
print(count_matrix[1:5,1:5])
write.csv(count_matrix,"amppd_chrY_featureCounts.csv",row.names=FALSE,quote=FALSE)

In [ ]:
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp amppd_chrY_featureCounts.csv {WORKSPACE_BUCKET}"))

## For Querying Specific Transcripts from Salmon

In [ ]:
#get the transcript we want
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/chrY_ids.txt ."))


In [ ]:
chry <- fread("chrY_ids.txt")
print(dim(chry))

chry <- chry[(chry$'hg19.knownToEnsembl.value'!='n/a'),]
print(dim(chry))
print(head(chry))

In [ ]:
y_transcripts <- chry$'hg19.knownToEnsembl.value'
print(length(y_transcripts))
print(length(unique(y_transcripts)))
y_transcripts <- unique(y_transcripts)
print(y_transcripts[1:10])

In [ ]:
samples_use$sample_id <- paste0(samples_use$iid,'-BLM0T1')


In [ ]:
#make a string to use in the query to get the samples we want
query_samples_string <- (paste(shQuote(c(samples_use$sample_id),type="cmd"),collapse=", "))
print(query_samples_string)

In [ ]:
#we are missing the ".#" part of the transcript names so need to account for this in the query. Unfortunately it seems the best way to do this is with many OR LIKE statements
query_transcripts_like <- paste(y_transcripts,collapse='.%" OR Name LIKE "')
query_transcripts_like <- paste0('Name LIKE "', query_transcripts_like, '.%"')

In [ ]:
#query_results <- bq_query(str_glue("
#  SELECT participant_id, Name, TPM, NumReads, EffectiveLength
#  FROM `amp-pd-research.2021_v2_5release_0510_transcriptomics.quantification_transcripts`
#  WHERE sample_id IN ({query_samples_string}) AND Name IN ('ENST00000454978.2','ENST00000382407.1')"))
query_results <- bq_query(str_glue("
  SELECT participant_id, Name, TPM, NumReads, EffectiveLength
  FROM `amp-pd-research.2021_v2_5release_0510_transcriptomics.quantification_transcripts`
  WHERE sample_id IN ({query_samples_string}) AND ({query_transcripts_like})"))
print(dim(query_results))

In [ ]:
head(query_results)

In [ ]:
print(length(unique(query_results$participant_id)))
print(length(unique(query_results$Name)))

In [ ]:
query_results[query_results$Name=="ENST00000383070.1",]

In [ ]:
counts_df <- query_results %>% select("participant_id","Name","NumReads")

In [ ]:
counts_matrix <- pivot_wider(counts_df, names_from = participant_id, values_from = NumReads)
print(dim(counts_matrix))
print(counts_matrix[1:5,1:5])
write.csv(counts_matrix,"amppd_chry_counts.csv",row.names=FALSE,quote=FALSE)

In [ ]:
abundance_df <- query_results %>% select("participant_id","Name","TPM")
abundance_matrix <- pivot_wider(abundance_df, names_from = participant_id, values_from = TPM)
print(dim(abundance_matrix))
print(abundance_matrix[1:5,1:5])
write.csv(abundance_matrix,"amppd_chry_abundance.csv",row.names=FALSE,quote=FALSE)

In [ ]:
length_df <- query_results %>% select("participant_id","Name","EffectiveLength")
length_matrix <- pivot_wider(length_df, names_from = participant_id, values_from = EffectiveLength)
print(dim(length_matrix))
print(length_matrix[1:5,1:5])
write.csv(length_matrix,"amppd_chry_length.csv",row.names=FALSE,quote=FALSE)

In [ ]:
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp amppd_chry_counts.csv {WORKSPACE_BUCKET}"))
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp amppd_chry_abundance.csv {WORKSPACE_BUCKET}"))
shell_do(str_glue("gsutil -u {BILLING_PROJECT_ID} cp amppd_chry_length.csv {WORKSPACE_BUCKET}"))